In [1]:
import rasterio
from rasterio.features import rasterize
from shapely.geometry import shape
import geopandas as gpd
import numpy as np
from rasterio.enums import MergeAlg

In [4]:
# ------------------- INPUT -------------------
shp_path = r'C:\Users\taohuang\Downloads\westcoast_forests_mtbs.shp'        
output_tif = r'C:\Users\taohuang\Downloads\mtbs_perimeter_data\mtbs_overlap_count_500m.tif'
# Desired resolution
resolution = 500  # meters

In [5]:

# Read the shapefile
print("Reading shapefile...")
gdf = gpd.read_file(shp_path)

# Reproject to a metric CRS if needed (recommended: use a UTM zone or Albers Equal Area)
# Here we automatically choose an appropriate UTM zone based on centroid
gdf = gdf.to_crs(gdf.estimate_utm_crs())

# Get total bounds and create output grid
bounds = gdf.total_bounds  # [xmin, ymin, xmax, ymax]
xmin, ymin, xmax, ymax = bounds

# Calculate raster dimensions
width = int(np.ceil((xmax - xmin) / resolution))
height = int(np.ceil((ymax - ymin) / resolution))

# Create transform
transform = rasterio.transform.from_origin(xmin, ymax, resolution, resolution)

# Prepare geometries (as shapely objects) - rasterize expects iterable of (geom, value)
# We use value=1 for each polygon so they add up
geoms = [(geom, 1) for geom in gdf.geometry]

print(f"Rasterizing {len(gdf)} polygons to {width}x{height} grid at {resolution}m resolution...")

# Rasterize with merge algorithm = ADD (this counts overlaps!)
burned = rasterize(
    shapes=geoms,
    out_shape=(height, width),
    transform=transform,
    fill=0,                    # background value
    all_touched=False,         # use True if you want more inclusive burning
    merge_alg=MergeAlg.add,    # This is the key: adds 1 for each overlapping polygon
    dtype=rasterio.uint16      # sufficient for overlap counts (max 65535)
)

# Write output GeoTIFF
print("Writing output GeoTIFF...")
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=rasterio.uint16,
    crs=gdf.crs,
    transform=transform,
    compress='deflate',
    predictor=2,
    tiled=True,
    blockxsize=256,
    blockysize=256
) as dst:
    dst.write(burned, 1)
    dst.update_tags(
        DESCRIPTION="Number of overlapping MTBS fire perimeters per 100m pixel"
    )

print(f"Done! Output saved to: {output_tif}")
print(f"   Max overlap: {burned.max()}")
print(f"   Area with >=1 burn: {np.sum(burned > 0) * resolution * resolution / 1e6:.2f} km²")

Reading shapefile...
Rasterizing 1836 polygons to 1717x3693 grid at 500m resolution...
Writing output GeoTIFF...
Done! Output saved to: C:\Users\taohuang\Downloads\mtbs_perimeter_data\mtbs_overlap_count_500m.tif
   Max overlap: 6
   Area with >=1 burn: 118529.00 km²
